In [1]:
import pandas as pd

In [4]:
df_critics = pd.read_csv('critics-psql.csv', index_col='id')
df_films = pd.read_csv('films-psql.csv', index_col='id')
df_reviews = pd.read_csv('reviews-psql.csv', index_col='id')
df_directors = pd.read_csv('directors-psql.csv', index_col='id')
df_directions = pd.read_csv('directions-psql.csv', index_col='id')

In [13]:
# Top 3 of the films appearing the most frequently on the top ten
df_reviews['film_id'].value_counts().head(3)

1     49
0     47
33    35
Name: film_id, dtype: int64

In [14]:
df_reviews.head(3)

,critic_id,film_id,rank
id,,,
0,0,0,0
1,0,1,1
2,0,2,2


In [20]:
df = df_reviews.merge(df_directions, on='film_id')
df.head()

,critic_id,film_id,rank,director_id
0,0,0,0,0
1,3,0,1,0
2,4,0,5,0
3,7,0,1,0
4,8,0,3,0


In [19]:
# How many times does Terrence Malick appear is the top ten and for which film
df[df['director_id'] == 2]['film_id'].value_counts()

2      23
124     7
15      1
548     1
Name: film_id, dtype: int64

In [27]:
df_directors.head()

,first_name,last_name
id,,
0,David,Lynch
1,Wong,Kar-wai
2,Terrence,Malick
3,Edward,Yang
4,Jean-Luc,Godard


In [28]:
df = df.merge(df_directors, left_on='director_id', right_index=True)

In [30]:
df['full_name'] = df['first_name'] + " " + df['last_name']

In [32]:
# How many times does David Lynch appear is the top ten and for which film
df[df['full_name'] == 'David Lynch']['film_id'].value_counts()

0     47
80     1
Name: film_id, dtype: int64

In [34]:
df = df.merge(df_films, left_on='film_id', right_index=True)

In [37]:
df[df['full_name'] == 'David Lynch']['title'].value_counts()

Mulholland Drive    47
Inland Empire        1
Name: title, dtype: int64

In [41]:
df[df['full_name'] == 'Ethan Coen']['title'].value_counts()

No Country For Old Men        21
Inside Llewyn Davis           20
A Serious Man                  8
O Brother, Where Art Thou?     3
Name: title, dtype: int64

In [141]:
df.groupby('full_name')['title'].value_counts()

full_name                    title                                      
Abbas Kiarostami             Certified Copy                                  9
                             Ten                                             4
                             Like Someone In Love                            2
Abdellatif Kechiche          Blue Is the Warmest Color                       7
Abderrahmane Sissako         Timbuktu                                        9
                             Waiting for Happiness                           2
Adam Curtis                  The Century of the Self                         1
Adam McKay                   Anchorman: The Legend of Ron Burgundy           1
                             Step Brothers                                   1
                             Talladega Nights: The Ballad of Ricky Bobby     1
Agnieszka Holland            Burning Bush                                    1
Agnès Jaoui                  The Taste of Others          

In [133]:
(df.groupby('title')
 .apply(lambda x: x['full_name'])
 .groupby(level=0)
 .filter(lambda x: len(x.unique()) > 2)
 .groupby(level=0)
 .apply(lambda x: x.unique()))

title
7 Letters                             [Boo Junfeng, Eric Khoo, Jack Neo, K. Rajagopa...
Leviathan                             [Andrey Zvyagintsev, Lucien Castaing-Taylor, V...
Madagascar 3: Europe's Most Wanted           [Eric Darnell, Tom McGrath, Conrad Vernon]
Monsters, Inc.                              [Lee Unkrich, Pete Docter, David Silverman]
Name: full_name, dtype: object